In [1]:
import pandas as pd
import numpy as np

In [2]:
df_1 = pd.read_csv("../data/Flight delays/flight_delays_train.csv")
df_2 = pd.read_csv("../data/Flight delays/flight_delays_test.csv")

In [3]:
df_1.shape, df_2.shape

((100000, 9), (100000, 8))

In [4]:
df = pd.concat([df_1, df_2])

In [5]:
df["month"] = df["Month"].str.replace("c-", "").astype(int)
df["WeekDay"] = df["DayOfWeek"].str.replace("c-", "").astype(int)
df["MonthDay"] = df["DayofMonth"].str.replace("c-", "").astype(int)

In [6]:
df["is_weekend"] = (df["WeekDay"] == 6) | (df["WeekDay"] == 7)

In [7]:
def categorize_dist(distance):
    if distance <= 1500:
        return "short-haul"
    elif 1500 < distance <= 3000:
        return "medium-haul"
    else:
        return "long_haul"

In [8]:
df["category"] = df["Distance"].apply(categorize_dist)

In [9]:
df["time"] = pd.to_datetime(df["DepTime"], format="%H%M", errors="coerce").dt.time

In [10]:
df['time'] = df["time"].fillna("00:00")

In [11]:
df.drop(["Month", "DayofMonth", "DayOfWeek", "DepTime"], axis=1, inplace=True)

In [12]:
df.head()

,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,month,WeekDay,MonthDay,is_weekend,category,time
0,AA,ATL,DFW,732,N,8,7,21,True,short-haul,19:34:00
1,US,PIT,MCO,834,N,4,3,20,False,short-haul,15:48:00
2,XE,RDU,CLE,416,N,9,5,2,False,short-haul,14:22:00
3,OO,DEN,MEM,872,N,11,6,25,True,short-haul,10:15:00
4,WN,MDW,OMA,423,Y,10,6,7,True,short-haul,18:28:00
